In [2]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 3.0 MB/s eta 0:00:00


In [3]:
!pip install --upgrade emoji

In [4]:
import emoji

In [5]:
# emoji.EMOJI_DATA

In [6]:
emoji_dictionary={"0":"\u2764\uFE0F",
                  "1":":baseball:",
                  "2":":grinning_face_with_big_eyes:",
                  "3":":disappointed_face:",
                  "4": ":fork_and_knife:",
                  "5":":hundred_points:",
                  "6":":fire:",
                  "7":":face_blowing_a_kiss:",
                  "8":":chestnut:",
                  "9":":flexed_biceps:"
                  }

In [7]:
for e in emoji_dictionary.values():
  print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


In [62]:
import pandas as pd
import numpy as np

In [63]:
train = pd.read_csv('/content/train_emoji.csv',header=None)
test = pd.read_csv('/content/test_emoji.csv',header=None)

In [64]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [65]:
data  = train.values
print(data.shape)

(132, 4)


In [66]:
X_train=train[0]
Y_train =train[1]

X_test=test[0]
Y_test=test[1]

In [67]:
for i in range(5):
  print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [68]:
f=open('glove.6B.50d.txt',encoding='utf-8')


In [69]:
embeddings_index={}
cnt=0
for line in f:
  values=line.split()
  word=values[0]
  coefs=np.asarray(values[1:],dtype='float')
  # print(word,coefs)
  embeddings_index[word]=coefs
f.close()

In [70]:
emb_dim = embeddings_index['eat'].shape[0]
print(emb_dim)

50


In [71]:
def embedding_output(X):
  maxLen=10
  embedding_out = np.zeros((X.shape[0],maxLen,emb_dim))

  for ix in range(X.shape[0]):
    X[ix]=X[ix].split()
    for ij in range(len(X[ix])):
      embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]

  return embedding_out

In [72]:
embeddings_matrix_train = embedding_output(X_train)
embeddings_matrix_test = embedding_output(X_test)

<ipython-input-71-ee3f981e4d66>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix]=X[ix].split()
<ipython-input-71-ee3f981e4d66>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix]=X[ix].split()
<ipython-input-71-ee3f981e4d66>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix]=X[ix].split()
<ipython-input-71-ee3f981e4d66>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the c

In [73]:
print(X_train[0])

['never', 'talk', 'to', 'me', 'again']


In [74]:
print(embeddings_matrix_train.shape)
print(embeddings_matrix_test.shape)

(132, 10, 50)
(56, 10, 50)


In [75]:
from keras.utils import to_categorical

In [76]:
Y_train[0]

3

In [77]:
Y_train = to_categorical(Y_train,num_classes=5)
Y_test = to_categorical(Y_test,num_classes=5)

In [81]:
Y_train[0]

array([0., 0., 0., 1., 0.], dtype=float32)

In [82]:
from keras.models import Sequential
from keras.layers import *

In [83]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 64)                29440     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
 activation_3 (Activation)   (None, 5)                 0         
                                                                 
Total params: 29765 (116.27 KB)
Trainable params: 29765 (116.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop = EarlyStopping(monitor='val_acc',patience=10)
hist=model.fit(embeddings_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Epoch 1/100
2/2 [==============================] - 0s 240ms/step - loss: 0.0106 - accuracy: 1.0000 - val_loss: 1.5529 - val_accuracy: 0.6296
Epoch 2/100
2/2 [==============================] - 0s 77ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 1.5242 - val_accuracy: 0.6296
Epoch 3/100
2/2 [==============================] - 0s 61ms/step - loss: 0.0111 - accuracy: 1.0000 - val_loss: 1.5038 - val_accuracy: 0.6296
Epoch 4/100
2/2 [==============================] - 0s 53ms/step - loss: 0.0126 - accuracy: 1.0000 - val_loss: 1.4953 - val_accuracy: 0.6296
Epoch 5/100
2/2 [==============================] - 0s 62ms/step - loss: 0.0108 - accuracy: 1.0000 - val_loss: 1.4940 - val_accuracy: 0.6296
Epoch 6/100
2/2 [==============================] - 0s 40ms/step - loss: 0.0228 - accuracy: 0.9905 - val_loss: 1.5300 - val_accuracy: 0.5926
Epoch 7/100
2/2 [==============================] - 0s 39ms/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 1.5999 - val_accuracy: 0.6296
Epoch 8/100
2/2 [==

In [87]:
import tensorflow as tf

In [98]:
pred = model.predict(embeddings_matrix_test)
np.argmax(pred,axis=1)

2/2 [==============================] - 0s 7ms/step


array([4, 3, 3, 3, 2, 2, 1, 2, 4, 2, 1, 2, 0, 2, 1, 3, 2, 2, 4, 4, 3, 3,
       4, 3, 3, 1, 2, 0, 1, 2, 0, 1, 3, 2, 0, 2, 2, 4, 0, 2, 2, 0, 0, 0,
       2, 2, 2, 2, 0, 1, 3, 0, 3, 2, 3, 3])

In [102]:
# model.load_weights("best_model.h5")

In [103]:
model.evaluate(embeddings_matrix_test,Y_test)

2/2 [==============================] - 0s 12ms/step - loss: 2.2816 - accuracy: 0.5893


[2.2816452980041504, 0.5892857313156128]

**New Model**

In [107]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 10, 64)            29440     
                                                                 
 dropout_6 (Dropout)         (None, 10, 64)            0         
                                                                 
 lstm_8 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
 activation_5 (Activation)   (None, 5)                 0         
                                                                 
Total params: 62789 (245.27 KB)
Trainable params: 6278

In [108]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.h6",monitor='val_loss',verbose=True,save_best_only=True)
earlystop = EarlyStopping(monitor='val_acc',patience=10)
hist=model.fit(embeddings_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Epoch 1/100
2/2 [==============================] - 5s 968ms/step - loss: 1.6444 - accuracy: 0.1429 - val_loss: 1.6201 - val_accuracy: 0.0370
Epoch 2/100
2/2 [==============================] - 0s 47ms/step - loss: 1.5918 - accuracy: 0.2952 - val_loss: 1.6050 - val_accuracy: 0.1852
Epoch 3/100
2/2 [==============================] - 0s 67ms/step - loss: 1.5567 - accuracy: 0.3143 - val_loss: 1.5983 - val_accuracy: 0.1852
Epoch 4/100
2/2 [==============================] - 0s 47ms/step - loss: 1.5413 - accuracy: 0.4286 - val_loss: 1.5969 - val_accuracy: 0.2222
Epoch 5/100
2/2 [==============================] - 0s 65ms/step - loss: 1.5292 - accuracy: 0.3905 - val_loss: 1.6013 - val_accuracy: 0.2222
Epoch 6/100
2/2 [==============================] - 0s 47ms/step - loss: 1.4824 - accuracy: 0.4190 - val_loss: 1.6125 - val_accuracy: 0.2222
Epoch 7/100
2/2 [==============================] - 0s 63ms/step - loss: 1.4896 - accuracy: 0.4095 - val_loss: 1.6263 - val_accuracy: 0.2222
Epoch 8/100
2/2 [==

In [112]:
pred = model.predict(embeddings_matrix_test)
pred1=np.argmax(pred,axis=1)

2/2 [==============================] - 0s 10ms/step


In [110]:
model.evaluate(embeddings_matrix_test,Y_test)

2/2 [==============================] - 0s 10ms/step - loss: 2.2914 - accuracy: 0.6071


[2.291445016860962, 0.6071428656578064]

In [113]:
pred1

array([4, 3, 3, 2, 2, 2, 1, 2, 4, 2, 1, 2, 0, 3, 1, 3, 2, 2, 3, 4, 0, 0,
       4, 0, 3, 1, 2, 0, 1, 2, 0, 1, 0, 2, 0, 2, 2, 4, 4, 2, 1, 0, 0, 1,
       2, 2, 2, 2, 3, 3, 3, 0, 3, 2, 3, 3])

In [115]:
for i in range(30):
  print(' '.join(X_test[i]))
  print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
  print(emoji.emojize(emoji_dictionary[str(pred1[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😞
she got me a present
❤️
😃
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
⚾
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious ha ha
😃
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
😃
stop messing around
😞
😞
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
❤️
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
❤️
Why are you feeling bad
😞
😞
I am upset
😞
⚾
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
⚾
valentine day is near
❤️
😃
